# Tanager batch (all matched-filter modes)

Auto-discover Tanager scenes (radiance + surface reflectance pairs) under `SCENE_ROOT` and run every matched-filter mode (srf-column k=1/k=3, full-column, advanced k=3, jpl). Outputs are written into per-mode subfolders and suffixed filenames to keep runs separate.

In [ ]:

from pathlib import Path
import os, sys

NOTEBOOK_ROOT = Path.cwd().resolve()
REPO_ROOT = NOTEBOOK_ROOT
while not (REPO_ROOT / "scripts").exists() and REPO_ROOT.parent != REPO_ROOT:
    REPO_ROOT = REPO_ROOT.parent
if not (REPO_ROOT / "scripts").exists():
    raise RuntimeError("Could not locate repository root containing scripts directory.")

os.environ.setdefault("PYTHONPATH", str(REPO_ROOT))
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

DEM_PATH = next(
    (p for p in (REPO_ROOT.parent / "DEM_1Km/srtm30plus_v11_land.nc", REPO_ROOT / "DEM_1Km/srtm30plus_v11_land.nc") if p.exists()),
    REPO_ROOT.parent / "DEM_1Km/srtm30plus_v11_land.nc",
)
LUT_PATH = next(
    (p for p in (REPO_ROOT.parent / "LUTs/CH4_lut.hdf5", REPO_ROOT / "LUTs/CH4_lut.hdf5") if p.exists()),
    REPO_ROOT.parent / "LUTs/CH4_lut.hdf5",
)
SNR_REF = REPO_ROOT / "reference_snr" / "tanager" / "snr_reference_columnwise.npz"
SCENE_ROOT = REPO_ROOT / "test_data" / "tanager" / "GHG-plumes"
OUTPUT_ROOT = REPO_ROOT / "notebooks" / "outputs" / "tanager" / "all_modes"
WINDOW = (2100.0, 2450.0)

print("Repo root:", REPO_ROOT)
for label, path in [("DEM", DEM_PATH), ("LUT", LUT_PATH), ("SNR", SNR_REF), ("Scene root", SCENE_ROOT)]:
    print(f"{label}: {path}")
missing = [label for label, path in [("DEM", DEM_PATH), ("LUT", LUT_PATH), ("SNR", SNR_REF)] if not path.exists()]
if missing:
    raise FileNotFoundError(f"Missing required assets: {', '.join(missing)}")


In [ ]:

from typing import Iterable

# Auto-discover scenes: look for basic_radiance* and basic_sr* pairs in immediate subfolders
SCENE_CONFIGS = []
for scene_dir in sorted(SCENE_ROOT.glob("*")):
    if not scene_dir.is_dir():
        continue
    rad_files = sorted(scene_dir.glob("basic_radiance_hdf5__*.h5"))
    sr_files = sorted(scene_dir.glob("basic_sr_hdf5__*.h5"))
    if not rad_files or not sr_files:
        continue
    SCENE_CONFIGS.append({
        "name": scene_dir.name,
        "rad": rad_files[0],
        "sr": sr_files[0],
    })

if not SCENE_CONFIGS:
    raise RuntimeError(f"No scenes found under {SCENE_ROOT}")
print(f"Discovered {len(SCENE_CONFIGS)} scene(s): {[c['name'] for c in SCENE_CONFIGS]}")

# mf_mode, k, suffix label
MF_RUNS: Iterable[tuple[str, int, str]] = (
    ("srf-column", 1, "srf-column_k1"),
    ("srf-column", 3, "srf-column_k3"),
    ("full-column", 1, "full-column"),
    ("advanced", 3, "advanced_k3"),
    ("jpl", 1, "jpl"),
)
print("MF runs:", [(m, k, s) for m, k, s in MF_RUNS])


In [ ]:

from scripts.pipelines import tanager_pipeline

OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
results = []

for cfg in SCENE_CONFIGS:
    for mf_mode, k, suffix in MF_RUNS:
        outdir = OUTPUT_ROOT / cfg["name"] / suffix
        outdir.mkdir(parents=True, exist_ok=True)
        print(f"Running {cfg['name']} [{mf_mode} k={k}] -> {outdir}")
        tanager_pipeline.ch4_detection_tanager(
            radiance_file=str(cfg["rad"]),
            sr_file=str(cfg["sr"]),
            dem_file=str(DEM_PATH),
            lut_file=str(LUT_PATH),
            output_dir=str(outdir),
            k=int(k),
            min_wavelength=WINDOW[0],
            max_wavelength=WINDOW[1],
            snr_reference_path=str(SNR_REF),
            mf_mode=mf_mode,
            output_name_suffix=suffix,
        )
        results.append(outdir)
print("Done.")


In [ ]:

from pathlib import Path
for outdir in results:
    print(f"Outputs in {outdir}:")
    for f in sorted(Path(outdir).glob("*")):
        print(" ", f.name)


In [ ]:

import rasterio
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

if not results:
    print("No runs completed; nothing to plot.")
else:
    last_outdir = Path(results[-1])
    conc = next(last_outdir.glob("*_MF.tif"), None)
    unc = next(last_outdir.glob("*_MF_uncertainty.tif"), None)
    if not conc or not unc:
        print("No output GeoTIFFs found to plot in", last_outdir)
    else:
        print("Plotting", conc.name, unc.name)
        with rasterio.open(conc) as src:
            conc_img = src.read(1)
        with rasterio.open(unc) as src:
            unc_img = src.read(1)

        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        im0 = axes[0].imshow(conc_img, cmap="inferno", vmin=np.nanpercentile(conc_img, 5), vmax=np.nanpercentile(conc_img, 95))
        axes[0].set_title("CH4 concentration (ppm·m)")
        plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)
        im1 = axes[1].imshow(unc_img, cmap="viridis", vmin=np.nanpercentile(unc_img, 5), vmax=np.nanpercentile(unc_img, 95))
        axes[1].set_title("σ_RMN (ppm·m)")
        plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
        for ax in axes:
            ax.axis("off")
        plt.tight_layout(); plt.show()
